# Análisis Semántico Latente
En este cuaderno se realizará la implementación del modelo de Analisis Semántico Latente (LSA) para dos documentos de prueba dentro del corpus.

Esta etapa esta compuesta de las siguientes fases:
1. Cargar datos.
2. Tokenizar por frases el corpus.
3. Aplicar el preprocesamiento al nuevo corpus.
4. Crear diccionarios para relacionar frases e identificadores.
5. Construir la matriz de A del Análisis Semántico Latente.
6. Aplicar la técnica de Análisis Semántico Latente.
7. Generación de resumen
8. Evaluar el resumen obtenido

In [1]:
#Importar elementos necesarios de las librerías
import os, shutil, re, pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rouge import Rouge
from nltk.corpus import PlaintextCorpusReader
from nltk.stem.snowball import SpanishStemmer
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [2]:
#Funciones auxiliares

def tokenize_sentence(path, file_name):
    #Retorna un documento tokenizado por frases
    doc = []
    text = PlaintextCorpusReader(path, file_name)
    paragraphs = text.paras()
    for paragraph in paragraphs:
        for sentence in paragraph:
            low, i = 0,0
            while i < len(sentence):
                token = sentence[i].split('.')
                if len(token)-1:
                    doc.append(sentence[low:i])
                    low=i+1
                    i+=2
                else:
                    i+=1
            if low!=i-1:
                doc.append(sentence[low:i])
    return doc

def preprocess(doc, stopwords, stemmer):
    #Aplica el preprocesamiento establecido
    #Adicionalmente, retorna el documento original sin las filas vacías por el preprocesamiento 
    doc_preprocesed, doc_reduced = [], []
    for original_sentence in doc:
        preprocessed_sentence = []
        for token in original_sentence:
            if stemmer.stem(token) not in stopwords:
                preprocessed_sentence.append(stemmer.stem(token))
        if len(preprocessed_sentence) and preprocessed_sentence not in doc_preprocesed:
            doc_preprocesed.append(preprocessed_sentence)
            doc_reduced.append(original_sentence)
    return doc_preprocesed, doc_reduced

def get_dictionaries(doc):
    #Retorna un par de diccionarios que relacionan una frase con un id, y un id con una frase.
    sentence2id, id2sentence = {},{}
    n_sentences = len(doc)
    for i in range(n_sentences):
        sentence = ' '.join(doc[i])
        if sentence not in sentence2id:
            sentence2id[sentence] = i
            id2sentence[i] = sentence
    return sentence2id, id2sentence

def getid2token(token2id):
    #Retorna un diccionario de tokens a id, a partir de un diccionario de id a tokens
    id2token ={}
    for k,v in token2id.items():
        id2token[v] = k
    return id2token

def build_A_Matrix(document, tf_idf, token2id, doc_id):
    #Construye la matriz A que recibe el modelo de LSA como entrada
    data,row_index,col_index = [],[],[]
    tf_idf = tf_idf.toarray()
    n,m = len(document), len(token2id)
    for i in range(n):
        sentence = document[i]
        j = 0
        for token in sentence:
            if token in token2id and tf_idf[doc_id,token2id[token]] != 0:
                if (j==0) or (j>0 and token2id[token] not in col_index[-j:]):
                    value = tf_idf[doc_id,token2id[token]]
                    tf_idf_value = value
                    data.append(tf_idf_value)
                    row_index.append(i)
                    col_index.append(token2id[token])
                    j+=1
    data = np.array(data)
    row_index = np.array(row_index)
    col_index = np.array(col_index)
    A_matrix = csr_matrix((data,(row_index,col_index)),shape=(n, m),dtype=np.float64)
    return A_matrix

def generate_summary(lsa, n_sentences, sentence2id):
    #Genera un resumen con n_sentences frases
    total_sentences = lsa.shape[0]
    assert n_sentences < total_sentences
    columns = ["topic {}".format(i) for i in range(n_sentences)]
    df = pd.DataFrame(lsa,columns=columns)
    df['sentence'] = sentence2id.keys()
    summary = []
    for i in range(n_sentences):
        df = df.sort_values(by='topic {}'.format(i), ascending = False)
        j = 0
        while j < total_sentences:
            sentence = df.iloc[j]['sentence']
            if sentence not in summary:
                summary.append(sentence)
                j=total_sentences
            else:
                j+=1
    return summary

def read_reference(path, filename):
    text = PlaintextCorpusReader(path, filename)
    text = text.paras()
    flat = []
    for sentences in text:
        for sentence in sentences:
            flat_sentence = ' '.join(sentence)
            flat.append(flat_sentence)
    reference = ' '.join(flat)
    return reference

## Fase 1. Cargar datos.
En esta fase se cargarán datos obtenidos durante el preprocesamientoy que utilizaremos para obtener el modelo de LSA.

In [3]:
#Cargar el modelo de tf-idf obtenido en preprocesamiento.
filename = 'tf-idf_model.pkl'
tf_idf = pickle.load(open(filename, 'rb'))

#Cargar la matriz de tf-idf obtenido en preprocesamiento.
filename = 'tf-idf_matrix.pkl'
tf_idf_matrix = pickle.load(open(filename, 'rb'))

#Cargar la matriz de tf-idf obtenido en preprocesamiento.
filename = 'stopwords.pkl'
stopwords = pickle.load(open(filename, 'rb'))

## Fase 2. Tokenizar las frases del corpus.
En esta fase se tokenizará por el corpus por frases.

In [4]:
ruta = "D:/Documents/Documentos Universidad/Noveno/Proyecto de grado/textos"
stemmer = SpanishStemmer()
corpus = []
referencias = ['azure-pagos.txt','chrome-privacidad.txt','colboletos.txt','gdo-privacidad.txt',
               'mozilla-privacidad.txt','nintendo-condiciones.txt','netflix-condiciones.txt',
               'segurosbolivar-privacidad.txt','whatsApp-privacidad.txt']
n_referencias = len(referencias)

for doc in referencias:
    documento = tokenize_sentence(ruta, doc)
    corpus.append(documento)

#colboletos = tokenize_sentence(ruta,'colboletos.txt')
#mozilla = tokenize_sentence(ruta,'mozilla-privacidad.txt')
#nintendo = tokenize_sentence(ruta,'nintendo-condiciones.txt')
bolivar = tokenize_sentence(ruta,'segurosbolivar-privacidad.txt')
#whatsapp = tokenize_sentence(ruta,'whatsApp-privacidad.txt')

## Fase 3. Aplicar el preprocesamiento al nuevo corpus.
En esta fase se aplicará el preprocesamiento establecido

In [5]:
corpus_preprocesado = []
for i in range(n_referencias):
    doc_preprocesado, corpus[i] = preprocess(corpus[i], stopwords, stemmer)
    corpus_preprocesado.append(doc_preprocesado)

bolivar_preprocesado, bolivar = preprocess(bolivar, stopwords, stemmer)
#mozilla_preprocesado, mozilla = preprocess(mozilla, stopwords, stemmer)
#colboletos_preprocesado, colboletos = preprocess(colboletos, stopwords, stemmer)
#nintendo_preprocesado, nintendo = preprocess(nintendo, stopwords, stemmer)
#whatsapp_preprocesado, whatsapp = preprocess(whatsapp, stopwords, stemmer)
#bolivar = corpus[i]

## Fase 4. Crear diccionarios para relacionar frases e identificadores.
Con el fin de obtener un resumen luego de aplicar la técnica de LSA, se necesita relacionar una frase con un identificador único. Esta fase cumple con dicho objetivo.

In [6]:
frase2id, id2frase = [],[]

for i in range(n_referencias):
    f2id, id2f = get_dictionaries(corpus[i])
    frase2id.append(f2id)
    id2frase.append(id2f)

#frase2id_colboletos, id2frase_colboletos = get_dictionaries(colboletos)
#frase2id_mozilla, id2frase_mozilla = get_dictionaries(mozilla)
#frase2id_nintendo, id2frase_nintendo = get_dictionaries(nintendo)
frase2id_bolivar, id2frase_bolivar = get_dictionaries(bolivar)
#frase2id_whatsapp, id2frase_whatsapp = get_dictionaries(whatsapp)

## Fase 5. Construir la matriz de A del Análisis Semántico Latente.
Hasta el momento, ya tenemos una lista de frases preprocesadas, ahora lo siguiente que tenemos que hacer es construir una matriz A de NxM, con N frases y M tokens. Para esto nos apoyaremos en el TF-IDF del preprocesamiento.

In [7]:
#Diccionario que relaciona tokens con un identificador único
token2id = tf_idf.vocabulary_

#Lista de textos dentro del corpus
textos = os.listdir(ruta)

matriz_A_corpus = []
for i in range(n_referencias):
    matriz_A_doc = build_A_Matrix(corpus_preprocesado[i], tf_idf_matrix, token2id, textos.index(referencias[i]))
    matriz_A_corpus.append(matriz_A_doc)

#matriz_A_colboletos = build_A_Matrix(colboletos_preprocesado, tf_idf_matrix, token2id, textos.index('colboletos.txt'))
#matriz_A_mozilla = build_A_Matrix(mozilla_preprocesado, tf_idf_matrix, token2id, textos.index('mozilla-privacidad.txt'))
#matriz_A_nintendo = build_A_Matrix(nintendo_preprocesado, tf_idf_matrix, token2id, textos.index('nintendo-condiciones.txt'))
matriz_A_bolivar = build_A_Matrix(bolivar_preprocesado, tf_idf_matrix, token2id, textos.index('segurosbolivar-privacidad.txt'))
#matriz_A_whatsapp = build_A_Matrix(whatsapp_preprocesado, tf_idf_matrix, token2id, textos.index('whatsApp-privacidad.txt'))

## Fase 6. Aplicar la técnica de Análisis Semántico Latente.
Una vez construida la matriz A, se utilizará la técnica de LSA para obtener una matriz de MxK, con M frases y K temas.

In [8]:
k = 5

#Instancia un modelo de LSA
lsa_model = TruncatedSVD(n_components=k)

#Aplica el LSA a la matriz tf-idf
lsa_corpus = []

for i in range(n_referencias):
    lsa_doc = lsa_model.fit_transform(matriz_A_corpus[i])
    lsa_corpus.append(lsa_doc)

#lsa_colboletos = lsa_model.fit_transform(matriz_A_colboletos)
#lsa_mozilla = lsa_model.fit_transform(matriz_A_mozilla)
#lsa_nintendo = lsa_model.fit_transform(matriz_A_nintendo)
lsa_bolivar = lsa_model.fit_transform(matriz_A_bolivar)
#lsa_whatsapp = lsa_model.fit_transform(matriz_A_whatsapp)
#matriz_A_bolivar=matriz_A_corpus[i]

In [9]:
#Dimensión de la matriz de Seguros Bolivar
#lsa_bolivar.shape

In [10]:
#Dimensión de la matriz de Mozilla
#lsa_mozilla.shape

## Fase 7. Generación del resumen
A partir del modelo de LSA obtenido en la fase anterior, se genera un resúmen escogiendo una frase por cada tema obtenido dentro del modelo de LSA. 

In [11]:
resumenes = []
for i in range(n_referencias):
    print(referencias[i])
    resumen_doc = generate_summary(lsa_corpus[i],k,frase2id[i])
    print(resumen_doc)
    print()
    resumenes.append(resumen_doc)

#resumen_bolivar = generate_summary(lsa_bolivar,k,frase2id_bolivar)
#resumen_bolivar

azure-pagos.txt
['Durante el Período de Vigencia de la Suscripción , los precios de los Servicios Online no aumentarán , en cuanto a la Suscripción , con relación a los publicados en el Portal en el momento en que la Suscripción entró en vigor o se renovó , salvo si los precios se identificaron como temporales en los Detalles de la Oferta o para Versiones Preliminares de Productos que no son de Microsoft', '( iii ) Para las Ofertas de Consumo , la Suscripción se renovará automáticamente para períodos de vigencia adicionales de un mes hasta que finalice la Suscripción', 'Usted es responsable del uso que haga el tercero de los Servicios de Microsoft Azure , de acuerdo con los términos de este contrato', 'Usted podrá utilizar el Producto solo de acuerdo con los términos de este contrato', 'Este Contrato Microsoft Online Subscription se celebra entre la entidad que usted representa o , en caso de que no designe ninguna entidad en relación con una adquisición o renovación de Suscripción , u

## Evaluar puntajes ROUGE

In [12]:
ruta_referencia = "D:/Documents/Documentos Universidad/Noveno/Proyecto de grado/referencias"

resumenes_referencia = []

for doc in referencias:
    referencia_doc = read_reference(ruta_referencia, doc)
    resumenes_referencia.append(referencia_doc)

#referencia_bolivar = read_reference(ruta_referencia, 'segurosbolivar-privacidad.txt')

#Castea cada resumen a un solo string
resumenes = [' '.join(doc) for doc in resumenes]

#resumen_bolivar = ' '.join(resumen_bolivar)
rouge = Rouge(metrics=["rouge-1", "rouge-l"])

In [13]:
for i in range(n_referencias):
    print(referencias[i])
    print(rouge.get_scores(resumenes[i],resumenes_referencia[i]))
    print()

azure-pagos.txt
[{'rouge-1': {'f': 0.30225080071817106, 'p': 0.7747252747252747, 'r': 0.1877496671105193}, 'rouge-l': {'f': 0.22281166731771848, 'p': 0.4421052631578947, 'r': 0.14893617021276595}}]

chrome-privacidad.txt
[{'rouge-1': {'f': 0.31685392952162617, 'p': 0.8392857142857143, 'r': 0.19529085872576177}, 'rouge-l': {'f': 0.3106266993233301, 'p': 0.6404494382022472, 'r': 0.20503597122302158}}]

colboletos.txt
[{'rouge-1': {'f': 0.4121739081977467, 'p': 0.3505917159763314, 'r': 0.5}, 'rouge-l': {'f': 0.17635270041084183, 'p': 0.17670682730923695, 'r': 0.176}}]

gdo-privacidad.txt
[{'rouge-1': {'f': 0.5276461248188996, 'p': 0.42710997442455245, 'r': 0.6900826446280992}, 'rouge-l': {'f': 0.4058823480249136, 'p': 0.359375, 'r': 0.46621621621621623}}]

mozilla-privacidad.txt
[{'rouge-1': {'f': 0.624113470322167, 'p': 0.7521367521367521, 'r': 0.5333333333333333}, 'rouge-l': {'f': 0.6304347776465029, 'p': 0.6904761904761905, 'r': 0.58}}]

nintendo-condiciones.txt
[{'rouge-1': {'f': 0.47